*tool - jupyter notebook (python=3.6)

# 단어의 표현 (Word Representation)
- 단어의 표현: 문자를 수치화하는 방법으로 자연어 처리에 필수
- Local Representation (통계 기반): 단어만을 보고 수치화
    + One-hot Encoding
    + N-gram
    + Count Based (BoW, TDM, TF-IDF)
- Distributed Representation (딥러닝 기반): 주변단어 참고하여 수치화
    + Prediction Based (Word2Vec, FastText)
    + Count Based (Windows-GloVe, Full Document-LSA)

## TDM (Term-Document Matrix) / DTM
- Bag of Words의 방법 중 하나
- 문서에 등장하는 각 단어의 등장빈도를 행렬로 표현
- 구현 방법
    + 1. 직접 구현하기
    + 2. sklearn : 통째로
    + 3. gensim : 쪼개서
- 단점
    + 단어의 순서 고려하지 않음
    + TDM 은 sparse > 벡터공간의 낭비, 연산 비효율성 초래
    + 단어 빈도수가 무조건 중요도 의미 X > TF-IDF 로 보완

## 1. 직접 구현하기

In [1]:
docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']

In [2]:
# 공백 토큰화
doc_ls = []
for doc in docs:
    doc_ls.append(doc.split(' '))
doc_ls

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [3]:
from collections import defaultdict
word2id = defaultdict(lambda : len(word2id))
for doc in doc_ls:
    for token in doc:
        word2id[token]        # token을 인덱스로 넣자
word2id

defaultdict(<function __main__.<lambda>()>,
            {'동물원': 0, '코끼리': 1, '원숭이': 2, '바나나': 3, '엄마': 4, '아기': 5})

In [4]:
doc_ls

[['동물원', '코끼리'],
 ['동물원', '원숭이', '바나나'],
 ['엄마', '코끼리', '아기', '코끼리'],
 ['원숭이', '바나나', '코끼리', '바나나']]

In [5]:
import numpy as np
TDM = np.zeros((len(word2id), len(doc_ls)), dtype = int)
for i, doc in enumerate(doc_ls):
    for token in doc:
        TDM[word2id[token],i] += 1       
# 행렬로 표기(BoW와 차이점: BoW는 1차원 배열)
TDM
# 실제 구현 코드는 여기까지

array([[1, 1, 0, 0],
       [1, 0, 2, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [6]:
# DataFrame화 및 시각화
import pandas as pd

doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
print('doc_names',doc_names)

sorted_vocab = sorted((value, key) for key, value in word2id.items())
vocab = [v[1] for v in sorted_vocab]
df_TDM = pd.DataFrame(TDM, columns = doc_names)
# df_TDM = pd.DataFrame(TDM, index = vocab, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

doc_names ['문서0', '문서1', '문서2', '문서3']


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
원숭이,0,1,0,1
바나나,0,1,0,2
엄마,0,0,1,0
아기,0,0,1,0


## 2. sklearn

In [7]:
docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# 참고 : sklearn은 DTM으로 만들어지게 설정되어 있음.
count_vect = CountVectorizer()
DTM = count_vect.fit_transform(docs)
DTM.toarray()

array([[1, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 2],
       [0, 2, 0, 0, 1, 1]], dtype=int64)

In [9]:
DTM.toarray().T

array([[1, 1, 0, 0],
       [0, 1, 0, 2],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 2, 1]], dtype=int64)

In [10]:
import pandas as pd

doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
vocab = count_vect.get_feature_names()
print(vocab)
df_TDM = pd.DataFrame(DTM.toarray().T, columns = doc_names)
# df_TDM = pd.DataFrame(TDM, index = vocab, columns = doc_names)
df_TDM['단어'] = vocab
df_TDM.set_index('단어')

['동물원', '바나나', '아기', '엄마', '원숭이', '코끼리']


,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
바나나,0,1,0,2
아기,0,0,1,0
엄마,0,0,1,0
원숭이,0,1,0,1
코끼리,1,0,2,1


## 3. gensim

In [11]:
docs = ['동물원 코끼리',
       '동물원 원숭이 바나나',
       '엄마 코끼리 아기 코끼리',
       '원숭이 바나나 코끼리 바나나']

In [12]:
import gensim
from gensim import corpora

doc_ls = [doc.split() for doc in docs] # 공백으로 토큰화

id2word = corpora.Dictionary(doc_ls)
TDM = [id2word.doc2bow(doc) for doc in doc_ls]
TDM
# 실제 코드 구현은 여기까지 하면 됨

[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1)],
 [(1, 2), (4, 1), (5, 1)],
 [(1, 1), (2, 2), (3, 1)]]

In [13]:
# 보기 편하게 sparse2full 사용(0이어서 안보이는 애를 0으로 채워줌)
from gensim.matutils import sparse2full
import pandas as pd
import numpy as np
doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
vocab = [id2word[i] for i in id2word.keys()]

DTM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in TDM]   # 출력 형태의 차이(list)

df_TDM = pd.DataFrame(np.array(DTM_matrix, dtype=int).T,
                     columns = doc_names)    # array형태로 변환해주어야 T 가능(df만들기 이전)

df_TDM['단어'] = vocab
df_TDM.set_index('단어')

,문서0,문서1,문서2,문서3
단어,,,,
동물원,1,1,0,0
코끼리,1,0,2,1
바나나,0,1,0,2
원숭이,0,1,0,1
아기,0,0,1,0
엄마,0,0,1,0


In [14]:
from gensim.matutils import sparse2full
import pandas as pd
import numpy as np
doc_names = ['문서' + str(i) for i in range(len(doc_ls))]
vocab = [id2word[i] for i in id2word.keys()]

DTM_matrix = [sparse2full(doc, len(vocab)) for doc in TDM]
df_DTM = pd.DataFrame(DTM_matrix, index = doc_names, columns=vocab).T
df_DTM

,문서0,문서1,문서2,문서3
동물원,1.0,1.0,0.0,0.0
코끼리,1.0,0.0,2.0,1.0
바나나,0.0,1.0,0.0,2.0
원숭이,0.0,1.0,0.0,1.0
아기,0.0,0.0,1.0,0.0
엄마,0.0,0.0,1.0,0.0
